In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


#### Data

In [43]:
csv_data = pd.read_csv('Audiobooks_data.csv',header=None)
csv_data.head()
target = csv_data.iloc[:,-1]

unbalanced_data = csv_data.iloc[:,1:-1]
csv_data.shape[0]
csv_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,994,1620.0,1620,19.73,19.73,1,10.00,0.99,1603.8,5,92,0
1,1143,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,0,0
2,2059,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,388,0
3,2882,1620.0,1620,5.96,5.96,0,8.91,0.42,680.4,1,129,0
4,3342,2160.0,2160,5.33,5.33,0,8.91,0.22,475.2,0,361,0


### Balancing the Data

In [51]:
row_to_remove = []
num_one_targets = int(np.sum(target))
num_zero = 0
for i in range(unbalanced_data.shape[0]):
    if target.iloc[i] == 0:
        num_zero +=1
        if num_zero > num_one_targets:
            row_to_remove.append(i)
            
unbalanced_data_equal_priors = unbalanced_data.drop(row_to_remove,axis=0)
targets_equal_priors = target.drop(row_to_remove,axis=0)
targets_equal_priors.value_counts()
        

1    2237
0    2237
Name: 11, dtype: int64

### Standardization of Data

In [52]:
standard_balanced_data = preprocessing.scale(unbalanced_data_equal_priors)

### Data Shuffling

In [60]:
shuffled_indices = np.arange(standard_balanced_data.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_data = standard_balanced_data[shuffled_indices]
shuffled_target = targets_equal_priors.iloc[shuffled_indices]

### Splitting the Data for Traning, Validation and Testing 

In [62]:
sample_count = shuffled_data.shape[0]

train_count = int(.8*sample_count)
validation_count = int(.1*sample_count)
test_sample_count = sample_count - train_count -validation_count

train_set = shuffled_data[:train_count]
train_target = shuffled_target.iloc[:train_count]

validation_set = shuffled_data[train_count:train_count+validation_count]
validation_target = shuffled_target.iloc[train_count:train_count+validation_count]

test_set = shuffled_data[train_count+validation_count:]
test_target = shuffled_target.iloc[train_count+validation_count:]

print(np.sum(train_target), train_count, np.sum(train_target)/train_count)
print(np.sum(validation_target), validation_count, np.sum(validation_target)/validation_count)
print(np.sum(test_target), test_sample_count, np.sum(test_target)/test_sample_count)

1794 3579 0.501257334450964
230 447 0.5145413870246085
213 448 0.47544642857142855


In [63]:
np.savez('train_dataset.npz', train_set,train_target)
np.savez('validation_dataset.npz',validation_set,validation_target)
np.savez('test_datset',test_set,test_target)